In [11]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression 


In [2]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [3]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://mayur-mlflow-artifacts/1', creation_time=1719952838464, experiment_id='1', last_update_time=1719952838464, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)


In [14]:
# Code with pipeline

with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)
    pipeline = make_pipeline(
        DictVectorizer(),
        LinearRegression()
    )
   
    
    
    
    pipeline.fit(dict_train, y_train)

    y_pred = pipeline.predict(dict_val)

    rmse= mean_squared_error(y_val, y_pred, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")
    

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 7.758715207983167


In [13]:
#Below code is without pipeline

with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)
    dv=DictVectorizer()
    
    
    X_train=dv.fit_transform(dict_train)
    
    X_val=dv.transform(dict_val)
    model = LinearRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    rmse= mean_squared_error(y_val, y_pred, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(model, artifact_path="model")
    with open('dict_vectorizer.bin', 'wb') as f_out:
        pickle.dump(dv,f_out)

    mlflow.log_artifact('dict_vectorizer.bin')

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 7.758715207983167


In [9]:
from mlflow.tracking import MlflowClient
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = 'c74235c34fb040ae99f608e425eb3614'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

In [21]:
client

In [22]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

In [8]:
pip install --upgrade jupyterlab ipywidgets
jupyter labextension update --all


SyntaxError: invalid syntax (1738952517.py, line 1)

In [25]:
jupyter labextension update --all


SyntaxError: invalid syntax (3780416829.py, line 1)